#카카오 API 사용 안하고 

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('37.55315 , 126.972533')
print(address)

서해바다, 391, 한강대로, 동자동, 남영동, 용산구, 서울특별시, 04320, 대한민국


In [13]:
address[0]

'서해바다, 391, 한강대로, 동자동, 남영동, 용산구, 서울특별시, 04320, 대한민국'

#카카오 api를 이용해서 reverse geocoding

In [14]:
# 카카오API를 사용하여 좌표->주소 변환
import requests, json, pprint

def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK 695ff44e2f92442949444abdd8ef53ee"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

full_address = get_address('37.55315', '126.972533')
pprint.pprint(full_address)


{'documents': [{'address_name': '서울특별시 용산구 동자동',
                'code': '1117010700',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '용산구',
                'region_3depth_name': '동자동',
                'region_4depth_name': '',
                'region_type': 'B',
                'x': 126.97239600694513,
                'y': 37.55205196070413},
               {'address_name': '서울특별시 용산구 남영동',
                'code': '1117053000',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '용산구',
                'region_3depth_name': '남영동',
                'region_4depth_name': '',
                'region_type': 'H',
                'x': 126.97336800305129,
                'y': 37.54284397575442}],
 'meta': {'total_count': 2}}


In [15]:
full_address

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '1117010700',
   'address_name': '서울특별시 용산구 동자동',
   'region_1depth_name': '서울특별시',
   'region_2depth_name': '용산구',
   'region_3depth_name': '동자동',
   'region_4depth_name': '',
   'x': 126.97239600694513,
   'y': 37.55205196070413},
  {'region_type': 'H',
   'code': '1117053000',
   'address_name': '서울특별시 용산구 남영동',
   'region_1depth_name': '서울특별시',
   'region_2depth_name': '용산구',
   'region_3depth_name': '남영동',
   'region_4depth_name': '',
   'x': 126.97336800305129,
   'y': 37.54284397575442}]}

In [18]:
import pandas as pd
df_subway = pd.read_csv('/content/drive/MyDrive/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv', encoding='cp949')

In [20]:
df_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            276 non-null    int64  
 1   호선            276 non-null    int64  
 2   고유역번호(외부역코드)  276 non-null    int64  
 3   역명            276 non-null    object 
 4   위도            276 non-null    float64
 5   경도            276 non-null    float64
 6   작성일자          276 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 15.2+ KB


In [28]:
df_filtered = df_subway[df_subway['역명'] == '시청']

In [29]:
df_filtered

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자
1,2,1,151,시청,37.563590,126.975407,1974-08-15
52,53,2,201,시청,37.563534,126.975271,1984-04-30


In [22]:
def get_dong(lat, lng):
    try:
        url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lng}&y={lat}"
        headers = {"Authorization": "KakaoAK 695ff44e2f92442949444abdd8ef53ee"}  # API 키 주의
        response = requests.get(url, headers=headers)

        if response.status_code == 200:  # 요청 성공
            result = response.json()
            for document in result['documents']:
                if document['region_type'] == 'B':  # region_type이 'B'인 경우
                    return document['region_3depth_name']
            return None  # 'B' 타입이 없는 경우
        else:
            return None  # 요청 실패 시 None 반환
    except Exception as e:
        print(f"Error: {e}")
        return None


In [23]:
df_subway[:2]

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자
0,1,1,150,서울,37.55315,126.972533,1974-02-28
1,2,1,151,시청,37.56359,126.975407,1974-08-15


In [30]:
# 샘플 데이터프레임
df = df_subway[:2]

# 각 행에 대해 '동' 정보를 추가
df_filtered['동'] = df_filtered.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)


<ipython-input-30-9efb321bc88c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['동'] = df_filtered.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)


In [31]:
df_filtered

,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자,동
1,2,1,151,시청,37.563590,126.975407,1974-08-15,서소문동
52,53,2,201,시청,37.563534,126.975271,1984-04-30,서소문동


In [ ]:
import pandas as pd
import requests
import json

# 카카오 API를 사용하여 주소를 가져오는 함수
def get_dong(lat, lng):
    try:
        url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lng}&y={lat}"
        headers = {"Authorization": "KakaoAK 695ff44e2f92442949444abdd8ef53ee"}  # API 키 주의
        response = requests.get(url, headers=headers)
        if response.status_code == 200:  # 요청 성공 시
            result = response.json()
            # 가장 상세한 행정 구역 정보 반환 ('동'에 해당)
            return result['documents'][0]['region_3depth_name']
        else:
            return None  # 요청 실패 시
    except Exception as e:
        print(f"Error: {e}")
        return None


# 각 행에 대해 '동' 정보를 추가
df_subway['동'] = df_subway.apply(lambda row: get_dong(row['위도'], row['경도']), axis=1)

In [ ]:
df_subway.head()

#카카오 api를 이용해서 지오코딩

In [3]:
# 카카오API를 사용하여 주소->좌표 변환
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 695ff44e2f92442949444abdd8ef53ee"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

crd = get_location("서울시 중구 을지로 234")
print(crd)

{'lat': '37.5664734170529', 'lng': '127.003943654119'}


In [4]:
# 카카오API를 사용하여 좌표->주소 변환
import requests, json, pprint

def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+lng+"&y="+lat
    headers = {"Authorization": "KakaoAK 695ff44e2f92442949444abdd8ef53ee"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

full_address = get_address('37.5664734170529', '127.003943654119')
pprint.pprint(full_address)


{'documents': [{'address_name': '서울특별시 중구 을지로5가',
                'code': '1114015100',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '중구',
                'region_3depth_name': '을지로5가',
                'region_4depth_name': '',
                'region_type': 'B',
                'x': 127.00243132639608,
                'y': 37.56660806668479},
               {'address_name': '서울특별시 중구 을지로동',
                'code': '1114060500',
                'region_1depth_name': '서울특별시',
                'region_2depth_name': '중구',
                'region_3depth_name': '을지로동',
                'region_4depth_name': '',
                'region_type': 'H',
                'x': 126.99135225472774,
                'y': 37.566701478579624}],
 'meta': {'total_count': 2}}
